# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](https://htmlpreview.github.io/?https://github.com/berserkhmdvhb/Data-Sciecne-with-Python/blob/main/Course1/Week4/Assignment/assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](https://github.com/berserkhmdvhb/Data-Sciecne-with-Python/blob/main/Course1/Week4/Assignment/assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](https://github.com/berserkhmdvhb/Data-Sciecne-with-Python/blob/main/Course1/Week4/Assignment/assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](https://github.com/berserkhmdvhb/Data-Sciecne-with-Python/blob/main/Course1/Week4/Assignment/assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](https://github.com/berserkhmdvhb/Data-Sciecne-with-Python/blob/main/Course1/Week4/Assignment/assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [152]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_ind
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [153]:
#MINE#
#Preparing NHL Dataset#
#1. citiesnhl
citiesnhl = cities.copy()
citiesnhl.drop(citiesnhl.columns[2:5], axis=1, inplace=True)
citiesnhl.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
##remove [note] from nhl column
citiesnhl['NHL'] = citiesnhl['NHL'].apply(lambda x: re.sub(r'\[[\w ]*\]','', x)) 
citiesnhl['NHL'] = citiesnhl['NHL'].apply(lambda x: re.sub(r'\—','', x)) 
##replace empty values with nan
nans = citiesnhl[citiesnhl['NHL'] == ''].index
citiesnhl['NHL'].iloc[nans] = np.nan

#2. nhldat
##select year 2018
nhldat = nhl_df[nhl_df['year'] == 2018]
##transform division from rows to a column corresponding to each team's division
nhldat['Division'] = 0
idx = 0
for i,tm in enumerate(nhldat['team']):
    name = tm
    if 'Division' in tm:
        nhldat['Division'].iloc[idx+1:i] = tm
        idx = i      
nhldat['Division'].iloc[idx+1:] = name
nhldat2=nhldat.copy()
tp = nhldat[nhldat['Division'] == 0]
nhldat=nhldat2[~nhldat2.index.isin(tp.index)]
df = nhldat.copy()
##remove asteriks
nhldat['team'] = nhldat['team'].apply(lambda x: x.strip('*')) 

#Find each NHL team's city in citiesnhl 
set1 = list(nhldat['team'].dropna())
set2 = list(citiesnhl['Metropolitan area'].dropna())
Dik={}
for s1 in set1:
    L = list(s1.split(' '))
    for s2 in set2:   
        if any(l in s2 for l in L):
            Dik[s1] = s2
rems = set(set1) - set(Dik.keys())
rems = list(rems)


#manually add missed city for teams
Dik['Arizona Coyotes'] = 'Phoenix'
Dik['Carolina Hurricanes'] = 'Raleigh'
Dik['Colorado Avalanche'] = 'Denver'
Dik['Florida Panthers'] = 'Miami–Fort Lauderdale'
Dik['Minnesota Wild'] = 'Minneapolis–Saint Paul'
Dik['Anaheim Ducks'] = 'Los Angeles'


##and, we must fix cities that include 'new' such as New York, New Jersery, ect.
Dik['New York Rangers'] = 'New York City'
Dik['New Jersey Devils'] = 'New York City'
Dik['New York Islanders'] = 'New York City'

##finally
Dik['San Jose Sharks'] = 'San Francisco Bay Area'

##lastly, we must fix cities that include 'bay' such as Tampa Bay, San Francisco Bay, etc.
Dik['Tampa Bay Lightning'] = 'Tampa Bay Area'

##creat a city column in nhldat
def findcity(x):
    if x in Dik:
        return Dik[x]
    else:
        return np.nan
nhldat['city'] = nhldat['team'].apply(findcity)      


##move city column to beginning
cols = nhldat.columns.tolist()
cols
cols.insert(1, cols[-1])
cols = cols[:-1]
nhldat = nhldat[cols]

nhldat.set_index('city', inplace=True)

#merge dataframes
citiesnhl.rename({'Metropolitan area': 'city'}, axis=1, inplace=True)
citiesnhl.set_index('city', inplace=True)

dfnhl = pd.merge(nhldat, citiesnhl, how='left', on='city')
dfnhl = dfnhl.reset_index()

##Sepratae dataframe to numeric and string columns

cols = dfnhl.columns.tolist()
cols
cols.insert(2, cols[-3])
cols.insert(3, cols[-1])
cols.insert(4, cols[-2])
cols = cols[:-4]
dfnhl = dfnhl[cols]

##turn numeric columns type to numeric
#convert columns types to numeric (instead of str)
for i in range (4,len(dfnhl.columns)):
    dfnhl[dfnhl.columns[i]] = pd.to_numeric(dfnhl[dfnhl.columns[i]])


In [154]:
rems

['Colorado Avalanche',
 'Anaheim Ducks',
 'Florida Panthers',
 'Arizona Coyotes',
 'Minnesota Wild',
 'Carolina Hurricanes']

In [155]:
#citiesnhl.sort_values('city').dropna()

In [156]:
#dfnhl.sort_values('city')

In [157]:
#dfnhl.sort_values('city')

In [158]:
#dfnhl.groupby('city').count()

In [159]:
L = []
for i,d in dfnhl.groupby('city'):
    L.append(d['Population'].values[0])

In [160]:
#len(dfnhlp['ratio'])

In [161]:
def nhl_correlation(): 
    # YOUR CODE HERE    
    global dfnhlp
    #removing nan vlues from dataframe based on population columns
    dfnhlp = dfnhl[dfnhl['Population'].notna()]
    dfnhlp['ratio'] = dfnhlp['W'] / (dfnhlp['W'] + dfnhlp['L'])
    dfnhlp = dfnhlp.groupby('city').agg('mean')
   
    #stats.pearsonr(population_by_region, win_loss_by_region)
    population_by_region = dfnhlp['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = dfnhlp['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return corr

In [162]:
nhl_correlation()

0.012486162921209907

In [163]:
#####dfnhlp[['Population','ratio']]

In [164]:
nba_df=pd.read_csv("assets/nba.csv")

In [165]:
#nba_df

In [166]:
nba_df=pd.read_csv("assets/nba.csv")

In [167]:
#MINE#
#Preparing NBA Dataset


#1. citiesnba
citiesnba = cities.copy()
citiesnba.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
##focus on NBA column (i=5)
i=4
citcols = citiesnba.columns
##remove [note] from NBA column
citiesnba[citcols[i]] = citiesnba[citcols[i]].apply(lambda x: re.sub(r'\[[\w ]*\]','', x)) 
citiesnba[citcols[i]] = citiesnba[citcols[i]].apply(lambda x: re.sub(r'\—','', x)) 
##replace empty values with nan
nans = citiesnba[citiesnba[citcols[i]] == ''].index
citiesnba[citcols[i]].iloc[nans] = np.nan


##only keep NBA 
L = list(citiesnba.columns[0:2])
L.append(citiesnba.columns[i])
citiesnba = citiesnba[L] 



#2.NBA
##select year 2018
nbadat = nba_df[nba_df['year'] == 2018]
##transform division from rows to a column corresponding to each team's division
nbadat['Division'] = 1
#idx = 0
#for i,tm in enumerate(nbadat['team']):
#    name = tm
#    if 'Division' in tm:
#        nbadat['Division'].iloc[idx+1:i] = tm
#        idx = i      
#nbadat['Division'].iloc[idx+1:] = name
#nbadat2=nbadat.copy()
#tp = nbadat[nbadat['Division'] == 0]
#nbadat=nbadat2[~nbadat2.index.isin(tp.index)]

##remove parentheses, asteriks and dash
nbadat['team'] = nbadat['team'].apply(lambda x: re.sub(r'[()\d]','', x)) 
nbadat['team'] = nbadat['team'].apply(lambda x: re.sub(r'\—','', x)) 
nbadat['team'] = nbadat['team'].apply(lambda x: re.findall(r'[\w ]*', x)[0]) 
#nbadat['Division'] = nbadat['Division'].apply(lambda x: re.sub(r'[()\d]','', x)) 

nbadat['GB'] = nbadat['GB'].apply(lambda x: re.sub(r'\—','', x)) 

##replace empty with nan
nbacols = nbadat.columns
nans = nbadat[nbadat['GB'] == ''].index
nbadat['GB'].loc[nans] = np.nan


##an exception
nbadat['team'][2] = 'Philadelphia 76ers'

#Find each NBA team's city in citiesnba 
set1 = list(nbadat['team'].dropna())
set2 = list(citiesnba['Metropolitan area'].dropna())
Dik={}
for s1 in set1:
    L = list(s1.split(' '))
    for s2 in set2:   
        if any(l in s2 for l in L):
            Dik[s1] = s2
rems = set(set1) - set(Dik.keys())
rems = list(rems)


#manually add missed city for teams
Dik['Brooklyn Nets'] = 'New York City'
Dik['Minnesota Timberwolves'] = 'Minneapolis–Saint Paul'
Dik['Utah Jazz'] = 'Salt Lake City'
Dik['Golden State Warriors'] = 'San Francisco Bay Area'


Dik['New Orleans Pelicans'] = 'New Orleans'
Dik['New York Knicks'] = 'New York City'
Dik['Oklahoma City Thunder'] = 'Oklahoma City'
Dik['San Antonio Spurs'] = 'San Antonio'




##create a city column in nbadat
def findcity(x):
    if x in Dik:
        return Dik[x]
    else:
        return np.nan
nbadat['city'] = nbadat['team'].apply(findcity)      


##move city column to beginning
cols = nbadat.columns.tolist()
cols
cols.insert(1, cols[-1])
cols = cols[:-1]
nbadat = nbadat[cols]

nbadat.set_index('city', inplace=True)

#merge dataframes
citiesnba.rename({'Metropolitan area': 'city'}, axis=1, inplace=True)
citiesnba.set_index('city', inplace=True)

dfnba = pd.merge(nbadat, citiesnba, how='left', on='city')
dfnba = dfnba.reset_index()

##Sepratae dataframe to numeric and string columns

cols = dfnba.columns.tolist()
cols
cols.insert(2, cols[-3])
cols.insert(3, cols[-1])
cols.insert(4, cols[-2])
cols = cols[:-4]
dfnba = dfnba[cols]

##turn numeric columns type to numeric
#convert columns types to numeric (instead of str)
for i in range (4,len(dfnba.columns)):
    dfnba[dfnba.columns[i]] = pd.to_numeric(dfnba[dfnba.columns[i]])


In [168]:
rems

['Utah Jazz',
 'Golden State Warriors',
 'Minnesota Timberwolves',
 'Brooklyn Nets']

In [169]:
#citiesnba.sort_values('city').dropna()

In [170]:
#dfnba.sort_values('city')

In [171]:
#dfnbap

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [172]:
def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    global dfnbap
    #removing nan vlues from dataframe based on population columns
    dfnbap = dfnba[dfnba['Population'].notna()]
    dfnbap['ratio'] = dfnbap['W'] / (dfnbap['W'] + dfnbap['L'])
    dfnbap = dfnbap.groupby('city').agg('mean')
        
    population_by_region = dfnbap['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = dfnbap['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    #stats.pearsonr(population_by_region, win_loss_by_region)
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return corr

In [173]:
nba_correlation()

-0.17657160252844617

In [174]:
mlb_df=pd.read_csv("assets/mlb.csv")

In [175]:
#MINE#
#Preparing MLB Dataset


#1. citiesmlb
citiesmlb = cities.copy()
citiesmlb.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
##focus on MLB column (i=5)
i=3
citcols = citiesmlb.columns
##remove [note] from MLB column
citiesmlb[citcols[i]] = citiesmlb[citcols[i]].apply(lambda x: re.sub(r'\[[\w ]*\]','', x)) 
citiesmlb[citcols[i]] = citiesmlb[citcols[i]].apply(lambda x: re.sub(r'\—','', x)) 
##replace empty values with nan
nans = citiesmlb[citiesmlb[citcols[i]] == ''].index
citiesmlb[citcols[i]].iloc[nans] = np.nan


##only keep MLB 
L = list(citiesmlb.columns[0:2])
L.append(citiesmlb.columns[i])
citiesmlb = citiesmlb[L] 



#2.MLB
##select year 2018
mlbdat = mlb_df[mlb_df['year'] == 2018]
##transform division from rows to a column corresponding to each team's division
mlbdat['Division'] = 1
#idx = 0
#for i,tm in enumerate(nbadat['team']):
#    name = tm
#    if 'Division' in tm:
#        nbadat['Division'].iloc[idx+1:i] = tm
#        idx = i      
#nbadat['Division'].iloc[idx+1:] = name
#nbadat2=nbadat.copy()
#tp = nbadat[nbadat['Division'] == 0]
#nbadat=nbadat2[~nbadat2.index.isin(tp.index)]

##remove doubledash
mlbdat['GB'] = mlbdat['GB'].apply(lambda x: re.sub(r'\--','', x)) 

##replace empty with nan
mlbcols = mlbdat.columns
nans = mlbdat[mlbdat['GB'] == ''].index
mlbdat['GB'].loc[nans] = np.nan



#Find each MLB team's city in citiesmlb 
set1 = list(mlbdat['team'].dropna())
set2 = list(citiesmlb['Metropolitan area'].dropna())
Dik={}
for s1 in set1:
    L = list(s1.split(' '))
    for s2 in set2:   
        if any(l in s2 for l in L):
            Dik[s1] = s2
rems = set(set1) - set(Dik.keys())
rems = list(rems)



In [176]:
mlbdat.sort_values('team')

,team,W,L,W-L%,GB,year,League,Division
27,Arizona Diamondbacks,82,80,0.506,9.5,2018,MLB,1
15,Atlanta Braves,90,72,0.556,NaN,2018,MLB,1
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,1
0,Boston Red Sox,108,54,0.667,NaN,2018,MLB,1
21,Chicago Cubs,95,68,0.583,1.0,2018,MLB,1
8,Chicago White Sox,62,100,0.383,29.0,2018,MLB,1
24,Cincinnati Reds,67,95,0.414,28.5,2018,MLB,1
5,Cleveland Indians,91,71,0.562,NaN,2018,MLB,1
26,Colorado Rockies,91,72,0.558,1.0,2018,MLB,1
7,Detroit Tigers,64,98,0.395,27.0,2018,MLB,1


In [177]:
rems

['Colorado Rockies',
 'Oakland Athletics',
 'Minnesota Twins',
 'Texas Rangers',
 'Arizona Diamondbacks']

In [178]:
Dik['Minnesota Twins'] = 'Minneapolis–Saint Paul'
Dik['Texas Rangers'] = 'Dallas–Fort Worth'
Dik['Oakland Athletics'] = 'San Francisco Bay Area'
Dik['Colorado Rockies'] = 'Denver'


Dik['New York Mets'] = 'New York City'
Dik['New York Yankees'] = 'New York City'
Dik['Kansas City Royals'] = 'Kansas City'

Dik['San Diego Padres'] = 'San Diego'
Dik['San Francisco Giants'] = 'San Francisco Bay Area'

Dik['Arizona Diamondbacks'] = 'Phoenix'

Dik['Tampa Bay Rays'] = 'Tampa Bay Area'
##create a city column in mlbdat
def findcity(x):
    if x in Dik:
        return Dik[x]
    else:
        return np.nan
mlbdat['city'] = mlbdat['team'].apply(findcity)      
a = mlbdat.copy()

##move city column to beginning
cols = mlbdat.columns.tolist()
cols
cols.insert(1, cols[-1])
cols = cols[:-1]
mlbdat = mlbdat[cols]

mlbdat.set_index('city', inplace=True)

#merge dataframes
citiesmlb.rename({'Metropolitan area': 'city'}, axis=1, inplace=True)
citiesmlb.set_index('city', inplace=True)

dfmlb = pd.merge(mlbdat, citiesmlb, how='left', on='city')
dfmlb = dfmlb.reset_index()

##Sepratae dataframe to numeric and string columns

cols = dfmlb.columns.tolist()
cols
cols.insert(2, cols[-3])
cols.insert(3, cols[-1])
cols.insert(4, cols[-2])
cols = cols[:-4]
dfmlb = dfmlb[cols]

##turn numeric columns type to numeric
#convert columns types to numeric (instead of str)
for i in range (4,len(dfmlb.columns)):
    dfmlb[dfmlb.columns[i]] = pd.to_numeric(dfmlb[dfmlb.columns[i]])



In [179]:
#citiesmlb.dropna().sort_values('city')

In [180]:
#dfmlb.sort_values('city')

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [181]:
def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    global dfmlbp
    #removing nan vlues from dataframe based on population columns
    dfmlbp = dfmlb[dfmlb['Population'].notna()]
    dfmlbp['ratio'] = dfmlbp['W'] / (dfmlbp['W'] + dfmlbp['L'])
    dfmlbp = dfmlbp.groupby('city').agg('mean')
    
    population_by_region = dfmlbp['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = dfmlbp['ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    #stats.pearsonr(population_by_region, win_loss_by_region)
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return corr

In [182]:
mlb_correlation()

0.15027698302669307

In [183]:
nfl_df=pd.read_csv("assets/nfl.csv")

In [184]:
#MINE#
#Preparing NFL Dataset


#1. citiesnfl
citiesnfl = cities.copy()
citiesnfl.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)
##focus on NFL column (i=5)
i=2
citcols = citiesnfl.columns
##remove [note] from NFL column
citiesnfl[citcols[i]] = citiesnfl[citcols[i]].apply(lambda x: re.sub(r'\[[\w ]*\]','', x)) 
citiesnfl[citcols[i]] = citiesnfl[citcols[i]].apply(lambda x: re.sub(r'\—','', x)) 
##replace empty values with nan
nans = citiesnfl[citiesnfl[citcols[i]] == ''].index
citiesnfl[citcols[i]].iloc[nans] = np.nan


##only keep NFL 
L = list(citiesnfl.columns[0:2])
L.append(citiesnfl.columns[i])
citiesnfl = citiesnfl[L] 


In [185]:



#2.NFL
##select year 2018
nfldat = nfl_df[nfl_df['year'] == 2018]
##transform division from rows to a column corresponding to each team's division
nfldat['Division'] = 0
idx = 0
for i,tm in enumerate(nfldat['team']):
    name = tm
    if 'NFC' in tm or 'AFC' in tm:
        nfldat['Division'].iloc[idx+1:i] = tm
        idx = i
nfldat['Division'].iloc[idx+1:] = name
nfldat2=nfldat.copy()
tp = nfldat[nfldat['Division'] == 0]
nfldat=nfldat2[~nfldat2.index.isin(tp.index)]


In [186]:
nfldat.sort_values('team')

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year,Division
39,-1.9,13,NFL,-12.5,-9.6,425,-200,225,-11.5,1.0,0,3,.188,Arizona Cardinals,2018,Arizona Cardinals
33,-2.6,9,NFL,-0.6,2.5,423,-9,414,-0.1,0.4,0,7,.438,Atlanta Falcons,2018,NFC West
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018,AFC South
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018,AFC North
32,0.8,9,NFL,-0.4,0.1,382,-6,376,0.9,1.3,0,7,.438,Carolina Panthers,2018,NFC West
26,4.8,4,NFL,8.6,1.5,283,138,421,6.3,-2.3,0,12,.750,Chicago Bears*,2018,NFC South
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Cincinnati Bengals,2018,AFC South
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Cleveland Browns,2018,AFC South
21,2.9,6,NFL,0.9,-1.9,324,15,339,1.1,0.2,0,10,.625,Dallas Cowboys*,2018,NFC North
18,3.1,10,NFL,-1.3,-3.6,349,-20,329,-0.5,0.7,0,6,.375,Denver Broncos,2018,NFC East


In [187]:
#remove asteriks and plus sign 
nfldat['team'] = nfldat['team'].apply(lambda x: re.findall(r'[\w ]*', x)[0]) 



#Find each NFL team's city in citiesnfl
set1 = list(nfldat['team'].dropna())
set2 = list(citiesnfl['Metropolitan area'].dropna())
Dik={}
for s1 in set1:
    L = list(s1.split(' '))
    for s2 in set2:   
        if any(l in s2 for l in L):
            Dik[s1] = s2
rems = set(set1) - set(Dik.keys())
rems = list(rems)



In [188]:
rems

['Oakland Raiders',
 'Carolina Panthers',
 'Tennessee Titans',
 'Minnesota Vikings',
 'Arizona Cardinals']

In [189]:
Dik['Tennessee Titans'] = 'Nashville'
Dik['Carolina Panthers'] = 'Charlotte'
Dik['Oakland Raiders'] = 'San Francisco Bay Area'
Dik['Arizona Cardinals'] = 'Phoenix'
Dik['Minnesota Vikings'] = 'Minneapolis–Saint Paul'

Dik['New England Patriots'] = 'Boston'
Dik['New Orleans Saints'] = 'New Orleans'
Dik['New York Giants'] = 'New York City'
Dik['New York Jets'] = 'New York City'

Dik['San Francisco 49ers'] = 'San Francisco Bay Area'

Dik['Green Bay Packers'] = 'Green Bay'
Dik['Tampa Bay Buccaneers'] = 'Tampa Bay Area'

Dik['Kansas City Chiefs'] = 'Kansas City'

##create a city column in nfldat
def findcity(x):
    if x in Dik:
        return Dik[x]
    else:
        return np.nan
nfldat['city'] = nfldat['team'].apply(findcity)      

##move city column to beginning
cols = nfldat.columns.tolist()
cols
cols.insert(1, cols[-1])
cols = cols[:-1]
nfldat = nfldat[cols]

nfldat.set_index('city', inplace=True)

#merge dataframes
citiesnfl.rename({'Metropolitan area': 'city'}, axis=1, inplace=True)
citiesnfl.set_index('city', inplace=True)

dfnfl = pd.merge(nfldat, citiesnfl, how='left', on='city')
dfnfl = dfnfl.reset_index()



##Sepratae dataframe to numeric and string columns

cols = dfnfl.columns.tolist()
cols

cols.insert(0, cols[14])
cols.insert(2, cols[-3])
cols.insert(3, cols[-1])
cols.insert(4, cols[-2])
cols.insert(4, cols[7])
cols = cols[:-5]
dfnfl = dfnfl[cols]


dfnfl.columns.get_loc('League')
dfnfl.drop(dfnfl.columns[8], axis=1, inplace=True)


##turn numeric columns type to numeric
#convert columns types to numeric (instead of str)
for i in range (4,len(dfnfl.columns)):
    dfnfl[dfnfl.columns[i]] = pd.to_numeric(dfnfl[dfnfl.columns[i]])


In [190]:
dfnflp = dfnfl[dfnfl['Population'].notna()]
dfnflp['ratio'] = dfnflp['W'] / (dfnflp['W'] + dfnflp['L'])
dfnflp = dfnflp.groupby('city').agg('mean')

In [191]:
dfnfl.shape

(32, 17)

In [192]:
dfnflp.shape

(29, 14)

In [193]:
#dfnflp.sort_values('city')

In [194]:
nfl_df[nfl_df['year']==2018].sort_values('team')

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
5,AFC North,AFC North,NFL,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,2018
10,AFC South,AFC South,NFL,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,2018
15,AFC West,AFC West,NFL,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,2018
39,-1.9,13,NFL,-12.5,-9.6,425,-200,225,-11.5,1.0,0,3,.188,Arizona Cardinals,2018
33,-2.6,9,NFL,-0.6,2.5,423,-9,414,-0.1,0.4,0,7,.438,Atlanta Falcons,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
32,0.8,9,NFL,-0.4,0.1,382,-6,376,0.9,1.3,0,7,.438,Carolina Panthers,2018
26,4.8,4,NFL,8.6,1.5,283,138,421,6.3,-2.3,0,12,.750,Chicago Bears*,2018


In [195]:
dfnfl.sort_values('team')

,team,city,Division,NFL,Population,DSRS,L,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%
31,Arizona Cardinals,Phoenix,Arizona Cardinals,Cardinals,4661537,-1.9,13,-12.5,-9.6,425,-200,225,-11.5,1.0,0,3,0.188
26,Atlanta Falcons,Atlanta,NFC West,Falcons,5789700,-2.6,9,-0.6,2.5,423,-9,414,-0.1,0.4,0,7,0.438
4,Baltimore Ravens,Baltimore,AFC South,Ravens,2798886,6.4,6,6.4,0.6,287,102,389,7.0,0.6,0,10,0.625
2,Buffalo Bills,Buffalo,AFC North,Bills,1132804,-0.6,10,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,0.375
25,Carolina Panthers,Charlotte,NFC West,Panthers,2474314,0.8,9,-0.4,0.1,382,-6,376,0.9,1.3,0,7,0.438
20,Chicago Bears,Chicago,NFC South,Bears,9512999,4.8,4,8.6,1.5,283,138,421,6.3,-2.3,0,12,0.750
7,Cincinnati Bengals,Cincinnati,AFC South,Bengals,2165139,-3.4,10,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,0.375
6,Cleveland Browns,Cleveland,AFC South,Browns,2055612,0.6,8,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,0.469
16,Dallas Cowboys,Dallas–Fort Worth,NFC North,Cowboys,7233323,2.9,6,0.9,-1.9,324,15,339,1.1,0.2,0,10,0.625
14,Denver Broncos,Denver,NFC East,Broncos,2853077,3.1,10,-1.3,-3.6,349,-20,329,-0.5,0.7,0,6,0.375


In [196]:
citiesnfl.sort_values('city')

,Population,NFL
city,,
Atlanta,5789700,Falcons
Baltimore,2798886,Ravens
Boston,4794447,Patriots
Buffalo,1132804,Bills
Calgary,1392609,NaN
Charlotte,2474314,Panthers
Chicago,9512999,Bears
Cincinnati,2165139,Bengals
Cleveland,2055612,Browns


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [197]:
def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    global dfnflp
    #removing nan vlues from dataframe based on population columns
    dfnflp = dfnfl[dfnfl['Population'].notna()]
    dfnflp['ratio'] = dfnflp['W'] / (dfnflp['W'] + dfnflp['L'])
    dfnflp = dfnflp.groupby('city').agg('mean')
        
    population_by_region = dfnflp['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = dfnflp['ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    #stats.pearsonr(population_by_region, win_loss_by_region)
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return corr

In [198]:
nfl_correlation()

0.004922112149349393

In [199]:
#dfnhlp
#dfmlbp
#ndfnbap
#dfnnflp

In [200]:
from functools import reduce 

In [201]:
print(dfnhl.shape)
print(dfmlb.shape)
print(dfnba.shape)
print(dfnfl.shape)

(31, 18)
(30, 10)
(30, 13)
(32, 17)


In [202]:
print(dfnhl.columns)
print(dfmlb.columns)
print(dfnba.columns)
print(dfnfl.columns)

Index(['city', 'team', 'Division', 'NHL', 'Population', 'GP', 'W', 'L', 'OL',
       'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'year'],
      dtype='object')
Index(['city', 'team', 'Division', 'MLB', 'Population', 'W', 'L', 'W-L%', 'GB',
       'year'],
      dtype='object')
Index(['city', 'team', 'Division', 'NBA', 'Population', 'W', 'L', 'W/L%', 'GB',
       'PS/G', 'PA/G', 'SRS', 'year'],
      dtype='object')
Index(['team', 'city', 'Division', 'NFL', 'Population', 'DSRS', 'L', 'MoV',
       'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS', 'T', 'W', 'W-L%'],
      dtype='object')


In [203]:
dfnhl['W-L%'] = dfnhl['W'] / (dfnhl['W'] + dfnhl['L'])
dfmlb.rename({'W/L%': 'W-L%'}, axis=1, inplace=True)
dfnba.rename({'W/L%': 'W-L%'}, axis=1, inplace=True)

In [204]:
print(dfnhl.columns)
print(dfmlb.columns)
print(dfnba.columns)
print(dfnfl.columns)

Index(['city', 'team', 'Division', 'NHL', 'Population', 'GP', 'W', 'L', 'OL',
       'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'year', 'W-L%'],
      dtype='object')
Index(['city', 'team', 'Division', 'MLB', 'Population', 'W', 'L', 'W-L%', 'GB',
       'year'],
      dtype='object')
Index(['city', 'team', 'Division', 'NBA', 'Population', 'W', 'L', 'W-L%', 'GB',
       'PS/G', 'PA/G', 'SRS', 'year'],
      dtype='object')
Index(['team', 'city', 'Division', 'NFL', 'Population', 'DSRS', 'L', 'MoV',
       'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS', 'T', 'W', 'W-L%'],
      dtype='object')


In [205]:
'{0}-{1}'.format(dfnhl.columns[3], dfmlb.columns[3])

'NHL-MLB'

In [206]:
def merge(df1, df2):
    data_frames = [df1.groupby('city').agg('mean'), df2.groupby('city').agg('mean')]
    df = reduce(lambda  left,right: pd.merge(left,right,on=['city'],
                                                how='inner'), data_frames)
    df.index.name = '{0}-{1}'.format(df1.columns[3], df2.columns[3])
    df = df[['W-L%_x', 'W-L%_y']]
    return df

In [207]:
def tstats(df1, df2): 
    df = merge(df1, df2)
    rat1 = df.iloc[:, [0]]
    rat2 = df.iloc[:, [1]]
    teststat,pval= stats.ttest_rel(rat1, rat2)
    name = df.index.name.split('-')
    p_values.loc['{0}'.format(name[0])]['{0}'.format(name[1])] = pval
    #return pval

In [208]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [209]:
sportsdf = [dfnhl, dfmlb, dfnba, dfnfl]

In [210]:
L=[]
for i in sportsdf:
    for j in sportsdf:
        df = merge(i,j)
        L.append(df.shape[0])
        print('{0}:{1}'.format(df.index.name, df.shape))
        #print('{0} shape is {1}'.format(a,b))

NHL-NHL:(28, 2)
NHL-MLB:(17, 2)
NHL-NBA:(14, 2)
NHL-NFL:(17, 2)
MLB-NHL:(17, 2)
MLB-MLB:(26, 2)
MLB-NBA:(18, 2)
MLB-NFL:(22, 2)
NBA-NHL:(14, 2)
NBA-MLB:(18, 2)
NBA-NBA:(28, 2)
NBA-NFL:(19, 2)
NFL-NHL:(17, 2)
NFL-MLB:(22, 2)
NFL-NBA:(19, 2)
NFL-NFL:(29, 2)


In [211]:
ans = sorted([29,19,17,22,19,28,14,18,17,14,28,17,22,18,17,26])
ans

[14, 14, 17, 17, 17, 17, 18, 18, 19, 19, 22, 22, 26, 28, 28, 29]

In [212]:
sorted(L)

[14, 14, 17, 17, 17, 17, 18, 18, 19, 19, 22, 22, 26, 28, 28, 29]

In [213]:
set(L) == set(ans)

True

In [214]:
set([1,2,3]) == set([3,1,2,2])

True

In [215]:
#for i in sportsdf:
 #   for j in sportsdf:
#        tstats(i,j)
 #       #print('{0} shape is {1}'.format(a,b))

In [216]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [217]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [218]:
for i in sportsdf:
    for j in sportsdf:
        tstats(i,j)


In [219]:
for i in range(4):
    p_values.iloc[i,i] = np.nan

In [220]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [221]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from functools import reduce 

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports) 
    #p_values.loc['{0}'.format(name[0])]['{0}'.format(name[1])] = pval
    #return pval
    for i in sportsdf:
        for j in sportsdf:
            tstats(i,j)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [222]:
p_values.loc["MLB", "NFL"]


0.8034592132146956

In [223]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
